In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [3]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from datetime import datetime

print(__version__) # requires version >= 1.9.0
import cufflinks as cf
# For Notebooks
init_notebook_mode(connected=True)
# For offline use
cf.go_offline()

4.1.1


In [4]:
data = pd.read_csv('./EMEAR_IT_CASES.csv',encoding = "ISO-8859-1", engine='python',parse_dates=['INCIDENT_OPENED_AT_DATE'])

In [5]:
data.shape

(84419, 34)

In [6]:
data.head(2)

,INCIDENT_NUMBER,INCIDENT_IMPACTED_USER,LABEL,ASSIGNEE_EMAIL,INCIDENT_SUMMARY,INCIDENT_NOTES,INCIDENT_STATE,IMPACTED_SERVICE,IMPACTED_SERVICE_OFFERING,ASSIGNMENT_GROUP,ASSIGNEMENT_SUPPORT_LEVEL,INCIDENT_CATEGORY,INCIDENT_SUBCATEGORY,FIRST_ASSIGNED_TIME,INCIDENT_OPENED_AT_DATE,INCIDENT_OPENED_FIS_MONTH,INCIDENT_OPENED_FIS_QUARTER,INCIDENT_OPENED_FIS_YEAR,INCIDENT_RESOLVED_AT_DATE,INCIDENT_CLOSED_AT_DATE,HR_COUNTRY,REGION,HR_THEATRE,DV_BUILDING,INCIDENT_RESOLVED_FIS_WEEK,INCIDENT_RESOLVED_FIS_MONTH,INCIDENT_RESOLVED_FIS_QUARTER,INCIDENT_RESOLVED_FIS_YEAR,INCIDENT_RESOLUTION_DOMAIN,INCIDENT_RESOLUTION_ACTION,INCIDENT_RESOLUTION_AREA,INCIDENT_RESOLUTION_NOTES,INCIDENT_IMPACTED_AREA,Priority
0,INC1603712,NaN,Monitoring,vinwilso@cisco.com,EUROPE1 - dbi03-lmr-gw1.cisco.com: Device not ...,EUROPE1 - dbi03-lmr-gw1.cisco.com: Device not ...,Cancelled,Corporate Network,Secure WAN Access,Network-Ops-Premise-NOC,Tier 2,Performance,Degraded,11-Jun-19,12-JUN-19 06.55.34.000000000,FY2019 MON 11 (JUN),FY2019 Q4,FY2019,NaN,12-JUN-19 07.10.15.000000000,UNITED ARAB EMIRATES,EMERGING,"EUROPE, MIDDLE EAST, AFRICA and RUSSIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 - Standard
1,INC2987458,francig3,Phone,mmartenc@cisco.com,Audio Issues (Poor Quality),*** Incident Reported ***\r\n\r\n Reported By...,Pending,Cloud Collaboration,Webex Teams,RCSS,Tier 1,Performance,Degraded,12-Jan-21,12-JAN-21 15.44.37.000000000,FY2021 MON 06 (JAN),FY2021 Q2,FY2021,NaN,NaN,PORTUGAL,EMEAR-SOUTH,"EUROPE, MIDDLE EAST, AFRICA and RUSSIA",OEIRAS 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voice - windows,5 - Standard


In [7]:
#import plotly.plotly as ply
import cufflinks as cf

#### It may not include all high incident as they are way high than what is in here

In [8]:
data.Priority.value_counts()

5 - Standard    81528
3 - Moderate     1520
4 - Minor        1262
1 - Critical       71
2 - High           38
Name: Priority, dtype: int64

In [9]:
# this data does not include all the high incidents
data[(data['INCIDENT_OPENED_FIS_QUARTER']=='FY2020 Q2')& (data['Priority'] =='2 - High')].groupby(['INCIDENT_OPENED_FIS_QUARTER','Priority','LABEL']).size().reset_index(name="Count").sort_values(by='Count', ascending=False)


,INCIDENT_OPENED_FIS_QUARTER,Priority,LABEL,Count
0,FY2020 Q2,2 - High,Phone,1
1,FY2020 Q2,2 - High,Portal,1


#### We can see howz the trend for SLA , how fast we are closing ? 

In [10]:
data.INCIDENT_STATE.value_counts()

Closed                 83330
Cancelled                495
Resolved                 288
Pending                  160
Work In Progress         114
Awaiting Assignment       31
Name: INCIDENT_STATE, dtype: int64

#### Mostly open incident are in current quarter

In [11]:
data[((data.INCIDENT_STATE=='Work In Progress' )| 
     (data.INCIDENT_STATE=='Awaiting Assignment') | (data.INCIDENT_STATE=='Pending')) & (data.INCIDENT_STATE != 'Cancelled')].INCIDENT_OPENED_FIS_QUARTER.value_counts().sort_index()

FY2018 Q4      1
FY2019 Q1      1
FY2019 Q2      3
FY2019 Q3     10
FY2019 Q4      7
FY2020 Q1      2
FY2020 Q2      5
FY2020 Q3      4
FY2020 Q4      3
FY2021 Q1     16
FY2021 Q2    253
Name: INCIDENT_OPENED_FIS_QUARTER, dtype: int64

In [12]:
data.LABEL.value_counts()

Phone              37577
Portal             23896
Walk-in             8063
Web                 5106
Chat                3878
Monitoring          3182
Email                967
Self-Service         878
Scheduled            710
Service Request      156
ServiceGrid            6
Name: LABEL, dtype: int64

In [13]:
dnew =data.groupby(['INCIDENT_OPENED_FIS_QUARTER','LABEL']).size().reset_index(name="Count").sort_values(by='INCIDENT_OPENED_FIS_QUARTER', ascending=False).reset_index()


In [14]:
dnew.head()

,index,INCIDENT_OPENED_FIS_QUARTER,LABEL,Count
0,136,FY2021 Q2,Web,9
1,135,FY2021 Q2,Walk-in,154
2,134,FY2021 Q2,Service Request,11
3,133,FY2021 Q2,Self-Service,198
4,132,FY2021 Q2,Scheduled,14


In [15]:
import plotly.express as px

fig = px.bar(dnew, x="INCIDENT_OPENED_FIS_QUARTER", y="Count", color='LABEL'
             #,
              #title='Alerts with Elvis policy distribution',
             , width=1000,height=400
               )  
    
 
#fig.update_xaxes(matches=None, showticklabels=True, visible=True)
fig.show()

In [16]:
data.INCIDENT_CATEGORY.value_counts()

Error              24537
Access             21565
Service Request    19733
Service/Process     9915
Performance         8233
Name: INCIDENT_CATEGORY, dtype: int64

In [20]:
#data.INCIDENT_CATEGORY.value_counts().iplot('bar')
dnew = data.groupby(['INCIDENT_OPENED_FIS_QUARTER','INCIDENT_CATEGORY']).size().reset_index(name="Count")


#### Service request related cases have gone down if we see below graph

In [22]:
import plotly.express as px

fig = px.bar(dnew, x="INCIDENT_OPENED_FIS_QUARTER", y="Count", color='INCIDENT_CATEGORY'
             #,
              #title='Alerts with Elvis policy distribution',
             , width=1000,height=400
               )  
    
 
#fig.update_xaxes(matches=None, showticklabels=True, visible=True)
fig.show()

In [17]:
### Looking by Incident subcategory
data.groupby(['INCIDENT_OPENED_FIS_QUARTER','INCIDENT_SUBCATEGORY']).size().reset_index(name="Count").sort_values(by='INCIDENT_OPENED_FIS_QUARTER',ascending=False).head(10)

,INCIDENT_OPENED_FIS_QUARTER,INCIDENT_SUBCATEGORY,Count
277,FY2021 Q2,User Call,185
268,FY2021 Q2,General Inquiry,1208
259,FY2021 Q2,Authentication,335
260,FY2021 Q2,Business Process,281
261,FY2021 Q2,Configuration,67
262,FY2021 Q2,Data,29
263,FY2021 Q2,Data or Configuration,1
265,FY2021 Q2,Degraded,318
266,FY2021 Q2,Document,1
267,FY2021 Q2,Function Not Working,133


In [18]:
#data.INCIDENT_SUBCATEGORY.value_counts().iplot('bar')
dnew = data.groupby(['INCIDENT_OPENED_FIS_QUARTER','INCIDENT_SUBCATEGORY']).size().reset_index(name="Count")


## Quick Points
###### General Inquiry,self service,Data has gone up in past 6 quarters
###### Business Process, Network,Authentication ,H/w & S/w related issue have gone down

In [19]:
import plotly.express as px

fig = px.bar(dnew, x="INCIDENT_OPENED_FIS_QUARTER", y="Count", color='INCIDENT_SUBCATEGORY'
             #,
              #title='Alerts with Elvis policy distribution',
             , width=1000,height=400
               )  
    
 
#fig.update_xaxes(matches=None, showticklabels=True, visible=True)
fig.show()